In [ ]:
!ls -al

In [ ]:
import torch

print(torch.__version__)

In [ ]:
!pwd

--Preprosessing

In [1]:
from datasets import load_dataset
train_dataset = load_dataset("jfleg", split='validation[:]')

eval_dataset = load_dataset("jfleg", split='test[:]')

Using custom data configuration default
Reusing dataset jfleg (/home/intern/.cache/huggingface/datasets/jfleg/default/1.0.0/165d64b6ad479919c1e2e24b0b4e8633d4ae76b335a477c6a4f89056eb5bcf07)
Using custom data configuration default
Reusing dataset jfleg (/home/intern/.cache/huggingface/datasets/jfleg/default/1.0.0/165d64b6ad479919c1e2e24b0b4e8633d4ae76b335a477c6a4f89056eb5bcf07)


In [2]:
import csv

def generate_csv(csv_path, dataset):
    with open(csv_path, 'w', newline='') as csvfile:
        writter = csv.writer(csvfile)
        writter.writerow(["input", "target"])
        for case in dataset:
     	    # Adding the task's prefix to input 
            input_text = case["sentence"]
            for correction in case["corrections"]:
                # a few of the cases contain blank strings. 
                if input_text and correction:
                    writter.writerow([input_text, correction])
                    


generate_csv("train.csv", train_dataset)
generate_csv("eval.csv", eval_dataset)

In [ ]:
for case in train_dataset["corrections"][:2]:
    print(case)

--testing

In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration, BartConfig

model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')

ARTICLE_TO_SUMMARIZE = "My friends are cool but they eat too many carbs."
inputs = tokenizer([ARTICLE_TO_SUMMARIZE], max_length=1024, return_tensors='pt')

summary_ids = model.generate(inputs['input_ids'], num_beams=4, max_length=5, early_stopping=True)
print([tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids])

In [5]:
import pandas as pd
train_path = '/home/intern/seungjun/error_data/train.csv'
test_path = '/home/intern/seungjun/error_data/eval.csv'

train_data = pd.read_csv(train_path, encoding='utf-8')
test_data = pd.read_csv(test_path, encoding='utf-8')

train_data.dropna(axis=0, inplace=True)
test_data.dropna(axis=0, inplace=True)

train_input = train_data['input'].tolist()
train_label = train_data['target'].tolist()

test_input = test_data['input'].tolist()
test_label = test_data['target'].tolist()

print(len(train_input))
print(len(train_label))

3016
3016


--BartConditionalGeneration

In [6]:
import torch
from transformers import BartTokenizer, BartForConditionalGeneration
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')

train_inputs_tokenize = tokenizer(train_input, return_tensors='pt', padding=True)
train_labels_tokenize = tokenizer(train_label, return_tensors='pt', padding=True)

for labels in train_labels_tokenize['input_ids']:
    for i in range(len(labels)):
        if labels[i]==tokenizer.pad_token_id:
            labels[i] = -100

print(train_inputs_tokenize['input_ids'].size())
print(train_labels_tokenize['input_ids'].size())

torch.save(train_inputs_tokenize, '/home/intern/seungjun/error_data/bart/training_input.pkl')
torch.save(train_labels_tokenize, '/home/intern/seungjun/error_data/bart/training_label.pkl')

torch.Size([3016, 90])
torch.Size([3016, 87])


In [6]:
import torch
from transformers import BartTokenizer, BartForConditionalGeneration, BartConfig
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')

test_inputs_tokenize = tokenizer(test_input, return_tensors='pt', padding=True)
test_labels_tokenize = tokenizer(test_label, return_tensors='pt', padding=True)

for labels in test_labels_tokenize['input_ids']:
    for i in range(len(labels)):
        if labels[i]==tokenizer.pad_token_id:
            labels[i] = -100



torch.save(test_inputs_tokenize, '/home/intern/seungjun/error_data/bart/evaluating_input.pkl')
torch.save(test_labels_tokenize, '/home/intern/seungjun/error_data/bart/evaluating_label.pkl')

In [26]:
print(test_inputs_tokenize['input_ids'][10])
print(test_labels_tokenize['input_ids'][0])

tensor([    0, 28526,  3916,    35,  4337,   621,   782,     7,   216,    10,
          828,    59, 10638,  2156, 17874,  2156,  5444,  2156, 13144,     8,
          750,    11,   645,     7,  1413,    66,    11,  2313,   479,     2,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1])
tensor([   0, 4030,  806,   34,   57, 2942,    7, 2313,  479,    2, -100, -100,
        -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -10

In [3]:
from tqdm import tqdm
import torch.optim as optim
import torch
from transformers import BartTokenizer, BartForConditionalGeneration, BartConfig

model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn').to('cuda')
optimizer = optim.Adam(model.parameters(), lr=1e-5)

def training_bart(batch_size):
    total_train_loss =0
    train_inputs_tokenize = torch.load('/home/intern/seungjun/error_data/bart/training_input.pkl')
    train_labels_tokenize = torch.load('/home/intern/seungjun/error_data/bart/training_label.pkl')
    k=0
    for iter in tqdm(range(len(train_inputs_tokenize['input_ids'])//batch_size)):
        optimizer.zero_grad()
        temp_input_list1 = train_inputs_tokenize['input_ids'][iter*batch_size:(iter+1)*batch_size].to('cuda')
        temp_input_list2 = train_inputs_tokenize['attention_mask'][iter*batch_size:(iter+1)*batch_size].to('cuda')
        temp_label_list = train_labels_tokenize['input_ids'][iter*batch_size:(iter+1)*batch_size].to('cuda')  
        temp = model(temp_input_list1,temp_input_list2, labels=temp_label_list)
        total_train_loss +=temp.loss.item()
        if(k%100==0 and k!=0):
            print("loss: ", temp.loss.item())
            print("loss average: ", total_train_loss/100)
            total_train_loss =0
        temp.loss.backward()
        optimizer.step()     
        k+=1



In [4]:
epochs = 5

for epoch in range(epochs):
    training_bart(5)
    model.save_pretrained('/home/intern/seungjun/error_model/bartcond_'+str(epoch))

 17%|█▋        | 101/603 [00:13<01:04,  7.74it/s]

loss:  0.34295767545700073
loss average:  0.8015401601791382


 33%|███▎      | 201/603 [00:26<00:49,  8.09it/s]

loss:  0.868419885635376
loss average:  0.6400506694614887


 50%|████▉     | 301/603 [00:38<00:37,  8.05it/s]

...

 83%|████████▎ | 501/603 [01:02<00:12,  8.05it/s]

loss:  0.20434334874153137
loss average:  0.23671324051916598


100%|█████████▉| 601/603 [01:15<00:00,  8.06it/s]

loss:  0.11142513155937195
loss average:  0.2452688653022051


100%|██████████| 603/603 [01:15<00:00,  8.00it/s]


In [29]:
import torch
import torch.optim as optim
from transformers import BartTokenizer, BartForConditionalGeneration, BartConfig
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
model = BartForConditionalGeneration.from_pretrained('/home/intern/seungjun/error_model/bartcond_4').to('cuda')

def ErrorSolution(input_string):
    tokenized_input = tokenizer([input_string], return_tensors='pt').to('cuda')
    summary_ids = model.generate(tokenized_input['input_ids'], num_beams = 32)

    result_arr =[]
    result = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids]
    result_arr.extend(result)

    return result

In [27]:
import torch
import torch.optim as optim
from transformers import BartTokenizer, BartForConditionalGeneration, BartConfig
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
model = BartForConditionalGeneration.from_pretrained('/home/intern/seungjun/error_model/bartcond_4').to('cuda')

def ErrorSolution_bart(input_string):
    tokenized_input = tokenizer([input_string], return_tensors='pt').to('cuda')
    summary_ids = model.generate(tokenized_input['input_ids'], num_beams = 32, max_length = tokenized_input['input_ids'].size()[1]+10)

    result_arr =[]
    result = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids]
    result_arr.extend(result)

    return result


In [28]:
ErrorSolution_bart(" One person if do n't have good health that means so many things they could lost .")


["If a person does n't have good health , that means they could lose so many things they could never have gotten rid of ."]

In [40]:
import torch
import torch.optim as optim
import numpy as np
from tqdm import tqdm
from transformers import BartTokenizer, BartForConditionalGeneration, BartConfig
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
model = BartForConditionalGeneration.from_pretrained('/home/intern/seungjun/error_model/bartcond_4').to('cuda')
model.eval()



def evaluating_bart(batch_size):
    total_eval_loss =0
    eval_inputs_tokenize = torch.load('/home/intern/seungjun/error_data/bart/evaluating_input.pkl')
    eval_labels_tokenize = torch.load('/home/intern/seungjun/error_data/bart/evaluating_label.pkl')
    k=0
    for iter in tqdm(range(len(eval_inputs_tokenize['input_ids'])//batch_size)):
        temp_input_list1 = eval_inputs_tokenize['input_ids'][iter*batch_size:(iter+1)*batch_size].to('cuda')
        temp_input_list2 = eval_inputs_tokenize['attention_mask'][iter*batch_size:(iter+1)*batch_size].to('cuda')
        temp_label_list = eval_labels_tokenize['input_ids'][iter*batch_size:(iter+1)*batch_size].to('cuda')
        with torch.no_grad():  
            temp = model(temp_input_list1,temp_input_list2, labels=temp_label_list)
            loss = temp.loss
        total_eval_loss +=loss.item()

    avg_loss = total_eval_loss/ (len(eval_inputs_tokenize['input_ids'])//batch_size)
    print(" Validation Loss: {0:.2f}".format(avg_loss))



In [41]:
evaluating_bart(5)

100%|██████████| 597/597 [00:15<00:00, 39.59it/s]

 Validation Loss: 0.61
